In [1]:
import os
import editdistance

In [2]:
train_names = ['11', '15', '17', '17об', '18об', '19', '19об', '20', '20об', '21', '21об', '22', '23', '23об', '24', '24об', '25', '25об', '26', '26об', '27', '27об', '28', '28об', '29', '29об', '2об', '30об', '31', '31об', '32', '32об', '33', '33об', '34', '34об', '35', '35об', '36', '38', '38об', '39', '39об', '40', '40об', '41', '41об', '42', '42об', '43', '43об', '44', '45об', '46', '46об', '47', '47об', '48', '48об', '49', '49об', '50', '50об', '51', '51об', '52', '52об', '53', '53об', '54', '54об', '55', '55об', '56', '56об', '57', '57об', '58']     
val_names = ['58об', '59', '59об', '60', '60об']
test_names = ['61', '75', '75об', '7об', '95']

In [3]:
model_text_path = "../../Подстрочник/Дневник_С-К/438-1-219/Model_text"
human_text_path = "../../Подстрочник/Дневник_С-К/438-1-219/Human_text"

In [4]:
def get_cer_from_lines(human_lines, model_lines, skip_hashtag=True):
    # Убираем пустую строку в конце
    if human_lines[-1].strip() == "":
        human_lines = human_lines[:-1]
    if model_lines[-1].strip() == "":
        model_lines = model_lines[:-1]
    if len(human_lines) != len(model_lines):
        return None
    cer_sum = 0
    n_chars_gt = 0
    n_evaluated_lines = 0
    for human_line, model_line in zip(human_lines, model_lines):
        human_line = human_line.strip()
        model_line = model_line.strip()
        if skip_hashtag and "#" in human_line:
            continue
        if human_line[-1] == "$":
            human_line = human_line[:-1]
        if model_line[-1] == "$":
            model_line = model_line[:-1]
        #print(human_line,":", model_line)
        cer_sum += editdistance.eval(human_line, model_line)
        n_chars_gt += len(human_line)
        n_evaluated_lines += 1
        #print(cer_sum, n_chars_gt)
    return cer_sum, n_chars_gt, n_evaluated_lines

In [5]:
def get_cer(names, name_prefix="438-1-219 л"):
    cer_sum = 0
    n_chars_gt = 0
    n_evaluated_lines = 0
    for human_name in os.listdir(human_text_path):
        if human_name[len(name_prefix):-4] in names:
            with open(os.path.join(human_text_path, human_name), "r", encoding="utf-8-sig") as human_file:
                human_lines = human_file.readlines()
            with open(os.path.join(model_text_path, human_name), "r", encoding="utf-8-sig") as model_file:
                model_lines = model_file.readlines()
            cer_from_lines = get_cer_from_lines(human_lines, model_lines)
            if cer_from_lines is None:
                print(f"len(human_lines) != len(model_lines) for {human_name[len(name_prefix):-4]}")
            else:
                cer_sum += cer_from_lines[0]
                n_chars_gt += cer_from_lines[1]
                n_evaluated_lines += cer_from_lines[2]

    return {"cer": cer_sum / n_chars_gt, "n_evaluated_lines": n_evaluated_lines}

In [6]:
get_cer(train_names)

len(human_lines) != len(model_lines) for 25
len(human_lines) != len(model_lines) for 2об
len(human_lines) != len(model_lines) for 39об
len(human_lines) != len(model_lines) for 55


{'cer': 0.08114695105993196, 'n_evaluated_lines': 1440}

In [7]:
get_cer(val_names)

{'cer': 0.2970104369361401, 'n_evaluated_lines': 119}

In [8]:
get_cer(test_names)

{'cer': 0.23698435277382646, 'n_evaluated_lines': 74}